In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

In [3]:
soja = pd.read_csv('soybean.csv')
soja.shape

(683, 36)

In [4]:
# Formato de matriz
previsores = soja.iloc[:,1:35].values
classe = soja.iloc[:,35].values

In [5]:
# Transformação dos previsores categoricos em numéricos
labelencoder = LabelEncoder()

for x in range(34):
    previsores[:, x] = labelencoder.fit_transform(previsores[:, x])

In [6]:
# Transformação da classe categorica em numérico
labelencoder_classe = LabelEncoder()
classe_encoded = labelencoder_classe.fit_transform(classe)

In [7]:
# Transformação da classe numérica em binária
classe_dummy = to_categorical(classe_encoded)
classe_dummy

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
#Divisão da base de dados entre treinamento e teste(30% teste e 70% treinamento)
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores,
                                                                  classe_dummy,
                                                                  test_size= 0.3,
                                                                  random_state = 0)
X_teste

array([[2, 1, 3, ..., 2, 1, 2],
       [1, 3, 3, ..., 0, 0, 3],
       [1, 0, 2, ..., 0, 0, 3],
       ...,
       [2, 1, 1, ..., 2, 1, 2],
       [1, 1, 1, ..., 2, 1, 3],
       [1, 3, 1, ..., 1, 2, 0]], dtype=object)

In [10]:
# Verifica o tipo de dados do conjunto de treinamento
print(X_treinamento.dtype)  
print(X_teste.dtype)

object
object


In [11]:
#Converte para float32
X_treinamento = X_treinamento.astype('float32')
X_teste = X_teste.astype('float32')

In [14]:
# Criação da estrutura da rede neural com a classe Sequential (sequência de camadas)
modelo = Sequential()
# Primeira camada oculta, 5 neurônios, 4 neurônios de entrada
modelo.add(Dense(units = 64, input_dim=34, activation='relu'))
# Segunda camada oculta
modelo.add(Dense(units = 32, activation='relu'))
# Função softmax porque temos um problema de classificação com mais de duas classes 
# (é gerada uma probabilidade em cada neurônio)
modelo.add(Dense(units=19, activation='softmax'))
# Visualização da estrutura da rede neural
modelo.summary()

C:\Users\elber\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         2,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 19)             │           627 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,947 (19.32 KB)

 Trainable params: 4,947 (19.32 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Configuração dos parâmetros da rede neural (adam = algoritmo para atualizar os pesos e loss = cálculo do erro)
modelo.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
               metrics = ['accuracy'])
# Treinamento, dividindo a base de treinamento em uma porção para validação (validation_data)
modelo.fit(X_treinamento, y_treinamento, epochs = 100,
           validation_data = (X_teste, y_teste))

Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.0776 - loss: 3.3451 - val_accuracy: 0.2049 - val_loss: 2.7299
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2363 - loss: 2.6207 - val_accuracy: 0.3024 - val_loss: 2.4597
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3734 - loss: 2.3128 - val_accuracy: 0.3659 - val_loss: 2.2658
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4391 - loss: 2.0948 - val_accuracy: 0.4049 - val_loss: 2.1088
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4338 - loss: 1.9430 - val_accuracy: 0.4585 - val_loss: 1.9631
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5250 - loss: 1.7587 - val_accuracy: 0.4732 - val_loss: 1.8214
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5100 - loss: 1.6459 - val_accuracy: 0.5171 - val_loss: 1.6808
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5849 - loss: 1.4540 - val_accuracy: 0.5415 

In [19]:
# Previsões utilizando os registros de teste
y_pred = modelo.predict(X_teste)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


In [20]:
# Preparação para comparação entre previsões e rótulos reais
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_teste, axis=1)

In [21]:
# Matriz de confusão
matriz_confusao = confusion_matrix(y_true_classes, y_pred_classes)
print(matriz_confusao)

[[ 3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 25  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  1  0]
 [ 0  0 14  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  7  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0 18  0  0  0  0  0  0  1  0  3  0  0  0  0]
 [ 0  0  0  0  0  0 13  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  9  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  5  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  6  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  5  0  0  0  0  0  0  0]
 [ 0  4  0  0  0  3  0  0  0  0  0  0 19  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  3  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  6  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 26  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0

In [22]:
taxa_acerto = accuracy_score(y_true_classes, y_pred_classes)
taxa_acerto

0.9170731707317074

In [23]:
taxa_erro = 1 - taxa_acerto 
taxa_erro

0.08292682926829265